## Init 🍄

---

### Dependencies:

In [ ]:
import yaml
import re
from datetime import datetime
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [ ]:
edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")

### Functions:

In [ ]:
def resources(name:str) -> dict[str:str]:
    """Read xpath.yml file that contains xpath's to scrape"""

    with open("xpath.yml", "r") as f:
        content:dict[str:str] = yaml.safe_load(f)
        return content[name]

In [ ]:
def close_prompt(driver:object) -> None:
    """Close ad window"""

    button:str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [ ]:
def choose_category(category:str, driver:object) -> None:
    """Choose ad category"""

    driver.find_element(by=By.XPATH, value=category).click()

In [ ]:
def click_element(element:object, path:str) -> None:
    """Click element on the current document"""

    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [ ]:
def extract_description(element:object, path:str) -> str:
    """Extract ad description on the current document"""

    description:str = element.find_element(by=By.XPATH, value=path)
    return description

In [ ]:
def extract_feature(element:object, path:str) -> list[str]:
    """Extract some feature from list document based on xpath used."""

    feature = []

    try:
        feature:str = element.find_elements(by=By.XPATH, value=path)
        feature = [data_point.text for data_point in feature]
        return feature
    except Exception:
        feature.append("Web wasn't able to scrape the feature")

In [ ]:
def persistence(dataframe:pd.DataFrame) -> None:
    """Generate .csv file from data scraped"""

    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [ ]:
def get_len(features:list[list[str]]) -> bool:
    """
    Check if all features has the same length to create new df
    :param features: list of features extracted with scraping
    :return: a list with all lengths where first place is the len of the first element and the mostly accepted.
    """

    base_len = len(features[0])
    lengths = []
    lengths.append(base_len)
    for feature in features:
        if len(feature) == base_len:
            continue
        else:
             lengths.append(len(feature))
    return lengths

In [48]:
def check_len(lengths:list[int]) -> bool:
    for length in lengths[1:]:
        if length != lengths[0]:
            return False
        return True

## Extraction 🧲

----

### Sedan:

Variables pool.

In [ ]:
descriptions:list[str] = []
prices:list[int] = []
models:list[str] = []
years:list[int] = []
kilometers:list[int] = []
engines:list[str] = []

# seconds
DELAY:int = 2

In [ ]:
sedan = resources("cars_sedan")

Get web & close initial prompt.

In [ ]:
edge.get(sedan["url"])

In [ ]:
close_prompt(edge)

In [ ]:
elements = edge.find_elements(by=By.XPATH, value=sedan["ad"])

Extract descriptions.

In [ ]:
elements[0].click()
try:
    for element in range(len(elements)):
        dynamic_delay = WebDriverWait(edge, DELAY).until(EC.presence_of_element_located((By.XPATH, sedan["description"])))
        temp = extract_description(edge, sedan["description"])
        descriptions.append(temp.text)
        click_element(edge, sedan["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, sedan["exit"])

Extract prices & models.

In [ ]:
prices = extract_feature(edge, sedan["price"])
models = extract_feature(edge, sedan["model"])

Extract car year, engine & kilometers.

In [ ]:
years = extract_feature(edge, sedan["year"])
engines = extract_feature(edge, sedan["engine"])
kilometers = extract_feature(edge, sedan["kilometers"])

In [ ]:
lengths = [prices, models, years, engines, kilometers, descriptions]
get_len(lengths)

## Cleaning 🤿

### Functions:

In [ ]:
def parse_price(dataframe: pd.DataFrame, feature:str, pattern:str) -> None:
    """
    Parse financial features like prices. Works with $#,### or $#,###\n* pattern
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param pattern: feature pattern to transform
    """
    dataframe[feature] = dataframe [feature] \
        .str.extract(pattern) \
        .applymap(lambda price: int(price.replace(",","")))


In [ ]:
def remove_chars(dataframe: pd.DataFrame, feature:str, wildcards:str) -> pd.Series:
    """
    Remove escaped characters from features like indented large extracts of text
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param wildcards: escaped characters to remove
    :return: Serie without escaped characters
    """
    return (dataframe
    .apply(
        lambda row: row[feature]
        ,axis = 1
    )
    .apply(
        lambda extracted_f: re.sub(wildcards, r' ', extracted_f)
    )
    )


In [ ]:
PRICE_RG: str = r"^\$(?P<prices>\d+,\d+)?"
KM_REG: str = r"(?P<km>^\d+)"
CHARACTER_RG:str = '(\n|\r)+'

---

Remove noise elements from data list

In [ ]:
# engines.append("Gasolina")

In [ ]:
# years.remove("Ver más")

In [ ]:
# models.remove("Blue Book Encuentra24")

In [ ]:
# kilometers.remove("Gasolina")
# kilometers.insert(-2, "0")

In [ ]:
# # engines.pop(-1)

Create dataframe to process data extracted

In [46]:
lengths = [len(engines), len(models), len(years), len(kilometers), len(prices),len(descriptions)]

[41, 41, 41, 41, 41, 41]

In [38]:
df_temp = pd.DataFrame({
    "model": models,
    "year": years,
    "kilometers": kilometers,
    "engine": engines,
    "price": prices,
    "description": descriptions
})

In [39]:
df_temp

,model,year,kilometers,engine,price,description
0,Mitsubishi Mirage G4,2018,60000 km,Gasolina,"$8,500\n(Rebajado 3%)",Color : Blanco\nSe vende Mitsubishi G4 2018. A...
1,Kia Rio,2020,56000 km,Gasolina,"$10,500\n(Rebajado 2%)",Color : Gris\nSe vende Kia Río 2020. Automátic...
2,Mazda 3,2013,91260 km,Gasolina,"$6,500",Precio negociable: si\nColor : Rojo\nMotor 2.0...
3,Toyota Corolla,2009,195504 km,Híbrido,"$17,000",Garantía: Como es visto no hay garantía\nPreci...
4,Toyota iA,2018,66 km,Gasolina,"$11,000",Precio negociable: si\nColor : Azul\nSe vende ...
5,Nissan Versa,2020,28235 km,Gasolina,"$8,200\n(Rebajado 4%)",Precio negociable: si\nBonito Nissan Versa 202...
6,Kia Rio,2016,97911 km,Gasolina,"$7,500",Financiamiento: no\nPrecio negociable: si\nCol...
7,Volkswagen Jetta,2014,131400 km,Gasolina,"$7,450",Garantía: Como es visto no hay garantía\nFinan...
8,Mitsubishi Lancer,2015,60000 km,Gasolina,"$8,500\n(Rebajado 3%)",Precio negociable: si\nColor : Negro\nSe vende...
9,Nissan Sentra,2013,140000 km,Gasolina,"$6,200",Precio negociable: no\nDESCRIPCIÓN\nCALIFICACI...


Parse features type from dataframe

In [40]:
parse_price(df_temp, "price", PRICE_RG)

In [41]:
df_temp["kilometers"] = df_temp["kilometers"].str.extract(KM_REG).applymap(int)
df_temp["year"] = df_temp["year"].map(int)

In [42]:
df_temp["description"] = remove_chars(df_temp, "description", CHARACTER_RG)

In [43]:
df_temp

,model,year,kilometers,engine,price,description
0,Mitsubishi Mirage G4,2018,60000,Gasolina,8500,Color : Blanco Se vende Mitsubishi G4 2018. Au...
1,Kia Rio,2020,56000,Gasolina,10500,Color : Gris Se vende Kia Río 2020. Automático...
2,Mazda 3,2013,91260,Gasolina,6500,Precio negociable: si Color : Rojo Motor 2.0 S...
3,Toyota Corolla,2009,195504,Híbrido,17000,Garantía: Como es visto no hay garantía Precio...
4,Toyota iA,2018,66,Gasolina,11000,Precio negociable: si Color : Azul Se vende to...
5,Nissan Versa,2020,28235,Gasolina,8200,Precio negociable: si Bonito Nissan Versa 2020...
6,Kia Rio,2016,97911,Gasolina,7500,Financiamiento: no Precio negociable: si Color...
7,Volkswagen Jetta,2014,131400,Gasolina,7450,Garantía: Como es visto no hay garantía Financ...
8,Mitsubishi Lancer,2015,60000,Gasolina,8500,Precio negociable: si Color : Negro Se vende M...
9,Nissan Sentra,2013,140000,Gasolina,6200,Precio negociable: no DESCRIPCIÓN CALIFICACION...


Generate .csv file for persistence from cleaned dataframe

In [44]:
persistence(df_temp)